In [1]:
from tomocupy_stream import GPURecRAM
import numpy as np
import tifffile
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib.pyplot as plt
%matplotlib inline

# Download data with the direct link

In [ ]:
!wget -nc https://anl.box.com/shared/static/or8vlzdu07d8zwxvk50ihwghq39ide0o.npz

--2023-05-30 19:49:44--  https://anl.box.com/shared/static/or8vlzdu07d8zwxvk50ihwghq39ide0o.npz
Resolving anl.box.com (anl.box.com)... 74.112.186.144
Connecting to anl.box.com (anl.box.com)|74.112.186.144|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/or8vlzdu07d8zwxvk50ihwghq39ide0o.npz [following]
--2023-05-30 19:49:44--  https://anl.box.com/public/static/or8vlzdu07d8zwxvk50ihwghq39ide0o.npz
Reusing existing connection to anl.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://anl.app.box.com/public/static/or8vlzdu07d8zwxvk50ihwghq39ide0o.npz [following]
--2023-05-30 19:49:44--  https://anl.app.box.com/public/static/or8vlzdu07d8zwxvk50ihwghq39ide0o.npz
Resolving anl.app.box.com (anl.app.box.com)... 74.112.186.144
Connecting to anl.app.box.com (anl.app.box.com)|74.112.186.144|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://public.boxcloud.com/d/1/b

# Read data, dark and flat fields, projection angles, the rotation center

In [ ]:
mdict = np.load('or8vlzdu07d8zwxvk50ihwghq39ide0o.npz')
data = mdict['data']
dark = mdict['dark']
flat = mdict['flat']
theta = mdict['theta']
center = mdict['center']


# Data consists of 1501 projections of the size [1024x1360]. For convience data are saved as sinograms (first two dimensions are swapped)

In [ ]:
data.shape 

# Plot projections

In [ ]:
def plot_projections(sid=1):
    plt.imshow(data[:,sid],cmap='gray')
interact(plot_projections, sid = widgets.IntSlider(value=data.shape[1]//2,
                                               min=0,
                                               max=data.shape[1]-1,
                                               step=1))

# Plot sinograms

In [ ]:
def plot_sinogram(sid=1):
    plt.imshow(data[sid],cmap='gray')
interact(plot_sinogram, sid = widgets.IntSlider(value=data.shape[0]//2,
                                               min=0,
                                               max=data.shape[0]-1,
                                               step=1))

# Create a class for reconstruction. Minus logarithm is turned off, dark=0, flat=1 since the inital data is already preprocessed.

In [ ]:
cl = GPURecRAM.for_data_like(
    data=data,
    dark=dark,
    flat=flat,
    ncz=8,  # chunk size for GPU processing (multiple of 2), 
    rotation_axis=center,  # rotation center
    dtype="float32",  # computation type, note  for float16 n should be a power of 2
    fbp_filter='parzen',
    minus_log=False
)

# Reconstruct by Tomocupy (the non-command-line version)

In [ ]:
obj = cl.recon_all(data, dark, flat, theta)

In [ ]:
def plot_rec(sid=1):
    plt.imshow(obj[sid],cmap='gray',vmin=-0.005,vmax=0.01)
interact(plot_rec, sid = widgets.IntSlider(value=obj.shape[0]//2,
                                               min=0,
                                               max=obj.shape[0]-1,
                                               step=1))

# Reproject data

In [ ]:
data_reproj = cl.proj_all(obj,theta)

In [ ]:
def plot_projections(sid=1):
    plt.figure(figsize=(15,10))
    plt.subplot(1,2,1)
    plt.title('initial data')
    plt.imshow(data[:,sid],cmap='gray')
    plt.subplot(1,2,2)
    plt.title('reprojected data')
    plt.imshow(data_reproj[:,sid],cmap='gray')
interact(plot_projections, sid = widgets.IntSlider(value=data.shape[1]//2,
                                               min=0,
                                               max=data.shape[1]-1,
                                               step=1))

In [ ]:
def plot_sinogram(sid=1):
    plt.figure(figsize=(15,10))
    plt.subplot(1,2,1)
    plt.title('initial data')
    plt.imshow(data[sid],cmap='gray')
    plt.subplot(1,2,2)
    plt.title('reprojected data')
    plt.imshow(data_reproj[sid],cmap='gray')    
interact(plot_sinogram, sid = widgets.IntSlider(value=data.shape[0]//2,
                                               min=0,
                                               max=data.shape[0]-1,
                                               step=1))

# Reconstruct the object by using reprojected data

## We can use the ramp filter instead of the parzen filter since we want to keep high frequencies in images. So we recreate the reconstruction class

In [ ]:
cl = GPURecRAM.for_data_like(
    data=data,
    dark=dark,
    flat=flat,
    ncz=8,  # chunk size for GPU processing (multiple of 2), 
    rotation_axis=center,  # rotation center
    dtype="float32",  # computation type, note  for float16 n should be a power of 2
    fbp_filter='ramp',
    minus_log=False
)

In [ ]:
obj_reproj = cl.recon_all(data_reproj, dark, flat, theta)

## Adjust scaling

In [ ]:
obj_reproj*=np.linalg.norm(obj)/np.linalg.norm(obj_reproj)

In [ ]:
def plot_rec(sid=1):
    plt.figure(figsize=(15,10))
    plt.subplot(1,2,1)
    plt.title('initial reconstruction')
    plt.imshow(obj_reproj[sid],cmap='gray',vmin=-0.005,vmax=0.01)
    plt.colorbar(orientation='horizontal')
    plt.subplot(1,2,2)
    plt.title('reconstruction after reprojection')
    plt.imshow(obj[sid],cmap='gray',vmin=-0.005,vmax=0.01)
    plt.colorbar(orientation='horizontal')
interact(plot_rec, sid = widgets.IntSlider(value=obj.shape[0]//2,
                                               min=0,
                                               max=obj.shape[0]-1,
                                               step=1))

## Plot difference, note colorbar

In [ ]:
def plot_rec(sid=1):
    plt.figure(figsize=(15,10))
    plt.imshow(obj[sid]-obj_reproj[sid],cmap='gray',vmin=-0.0002,vmax=0.0002)    
    plt.colorbar(orientation='horizontal')
interact(plot_rec, sid = widgets.IntSlider(value=obj.shape[0]//2,
                                               min=0,
                                               max=obj.shape[0]-1,
                                               step=1))